In [ ]:
# Using readlines()
file1 = open('preds (1).txt', 'r')
Lines = file1.readlines()
 

preds = []
# Strips the newline character
for line in Lines:
  preds.append(line.strip())

In [ ]:
print(len(tweets))

231711


In [ ]:
file1 = open('db_tweets_v2.txt', 'r')
Lines = file1.readlines()
 

tweets = []
# Strips the newline character
for line in Lines:
  tweets.append(line.strip())

In [ ]:
ls

'db_annotated_tweets (1).csv'   democs.txt       repubs.txt
 db_tweets_v2.txt              'preds (1).txt'   sample_data/


In [ ]:
yz = []
for i in range(3000):
  yz.append(tweets[i])
  if preds[i] == "negative":
    yz.append(-1)
  else:
    yz.append(1)


In [ ]:
textfile = open("a_file_3000.txt", "w")
for element in yz:
    textfile.write(str(element) + "\n")
textfile.close()

In [ ]:
!pip install emoji

     |████████████████████████████████| 174 kB 7.1 MB/s 
  Created wheel for emoji: filename=emoji-1.6.3-py3-none-any.whl size=170298 sha256=c9574935c5e35a8205c349d49afabec7c54aec7ad90685310b52bc48f3bf301a
  Stored in directory: /root/.cache/pip/wheels/03/8b/d7/ad579fbef83c287215c0caab60fb0ae0f30c4d7ce5f580eade
Successfully built emoji


In [ ]:
import pandas as pd
from tqdm import tqdm
import re
import emoji
import nltk
tqdm.pandas()

In [ ]:
nltk.download('words')
words = set(nltk.corpus.words.words())
def cleaner(tweet):
    tweet = re.sub("@[A-Za-z0-9]+","",tweet) #Remove @ sign
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) #Remove http links
    tweet = " ".join(tweet.split())
    tweet = ''.join(c for c in tweet if c not in emoji.UNICODE_EMOJI) #Remove Emojis
    tweet = tweet.replace("#", "").replace("_", " ") #Remove hashtag sign but keep the text
    tweet = " ".join(w for w in nltk.wordpunct_tokenize(tweet) \
         if w.lower() in words or not w.isalpha())
    return tweet

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


In [ ]:
ents = pd.read_csv('entities.csv', index_col=False)
ents = ents.fillna('')

In [ ]:
tweets = pd.read_csv('db_annotated_tweets.csv', index_col=False)
tweets = tweets.fillna('')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
tweets['TweetText'] = tweets['TweetText'].map(lambda x: cleaner(x))
tweets['UserName'] = tweets['UserName'].apply(lambda x: x.replace('@', '').replace(' ', ''))

In [ ]:
import string
clean_ents = []

for ee in ents["interest"].values:
  ee = ee.lower()
  ee = ee.translate(str.maketrans('', '', string.punctuation))
  clean_ents.append(ee)

In [ ]:
file1 = open('repubs.txt', 'r')
Lines = file1.readlines()
 

repubs = []
ori_repubs = []
# Strips the newline character
for line in Lines:
  ori_repubs.append(line.strip())
  rr = line.strip().lower()
  rr = rr.replace("http://dbpedia.org/resource/","")
  rr = rr.replace("_"," ")
  rr = rr.translate(str.maketrans('', '', string.punctuation))
  repubs.append(rr)

ind_dict_repubs = dict((k,i) for i,k in enumerate(repubs))

In [ ]:
file2 = open('democs.txt', 'r')
Lines2 = file2.readlines()
 

democs = []
ori_democs = []
# Strips the newline character
for line in Lines2:
  ori_democs.append(line.strip())
  rr = line.strip().lower()
  rr = rr.replace("http://dbpedia.org/resource/","")
  rr = rr.replace("_"," ")
  rr = rr.translate(str.maketrans('', '', string.punctuation))
  democs.append(rr)

ind_dict_democs = dict((k,i) for i,k in enumerate(democs))

In [ ]:
print(len(repubs))
new_repubs = list(set(repubs) - set(clean_ents))
print(len(new_repubs))

29504
29046


In [ ]:
print(len(democs))
new_democs = list(set(democs) - set(clean_ents))
print(len(new_democs))

28535
28167


In [ ]:
repubs_filter = [ ori_repubs[ind_dict_repubs[x]] for x in new_repubs ]

In [ ]:
democs_filter = [ ori_democs[ind_dict_democs[x]] for x in new_democs ]

In [ ]:
import pickle
with open(r"argmax_pred.pickle", "rb") as input_file:
  preds = pickle.load(input_file)

print(len(preds))

3000


In [ ]:
file3 = open('a_file_3000.txt', 'r')
Lines3 = file3.readlines()
yz = []
# Strips the newline character
for line in Lines3:
  yz.append(line.strip())

yz_new = list(filter(lambda a: a != "1", yz))
yz_last = list(filter(lambda a: a != "-1", yz_new))

In [ ]:
len(yz_last)

3000

In [ ]:
tweets = tweets[tweets['TweetText'].isin(yz_last)]
tweets.shape

(462, 14)

In [ ]:
tweets.head()

,Unnamed: 0,index,Unnamed: 0.1,UserName,TweetText,Timestamp,Tweet URL,collector,TweetID,Entities,UserID,retweet,mentions,db_ents
2,2,2,2,dmrose15,to many are offended if you simply say yes or ...,2021-10-25T23:58:11.000Z,https://twitter.com/dmrose15/status/1452786599...,scraper,1.45279e+18,,,,"_p_rittenhouse,","""evening"",""TIME"","""","""","""""
18,18,18,18,_richard_ong_,Trump . is truly up . How are going to parade ...,2021-10-25T23:25:31.000Z,https://twitter.com/_richard_ong_/status/14527...,scraper,1.45278e+18,,,,,"""Donald Trump Jr."",""DBPEDIA_ENT"",""http://dbped..."
19,19,19,19,blizthehusky,hey why ' t Kyle in jail ? insider . Trump . i...,2021-10-25T23:25:29.000Z,https://twitter.com/blizthehusky/status/145277...,scraper,1.45278e+18,,,,"GOP,","""Kyle"",""DBPEDIA_ENT"",""http://dbpedia.org/resou..."
21,21,21,21,TaylorKirkham,"to and 3 "" fired ""... do you want to clarify o...",2021-10-25T23:23:42.000Z,https://twitter.com/TaylorKirkham/status/14527...,scraper,1.45278e+18,,,,"eriksgrossman26,;steverasch,","""shots fired"",""DBPEDIA_ENT"",""http://dbpedia.or..."
27,27,27,27,GrayscaleLyric,in the middle of broke,2021-10-25T23:19:04.000Z,https://twitter.com/GrayscaleLyric/status/1452...,scraper,1.45278e+18,,,,,


In [ ]:
import random
ent_ext = []
j = 0
for tweet in tweets['TweetText'].values:
  tmp = []
  index = yz_last.index(tweet)
  if preds[index] == 2:
    sampled_list = random.sample(repubs_filter, 2)
  else:
    sampled_list = random.sample(democs_filter, 2)

  tmp.append(tweets['UserName'].values[j])
  tmp.append("Person")
  tmp.append(sampled_list[0])
  ent_ext.append(tmp)
  tmp = []
  tmp.append(tweets['UserName'].values[j])
  tmp.append("Person")
  tmp.append(sampled_list[1])
  ent_ext.append(tmp)
  j = j+1

new_ents = pd.DataFrame(ent_ext, columns=["user","interest type", "interest"])


In [ ]:
new_ents.head(5)

,user,interest type,interest
0,dmrose15,Person,http://dbpedia.org/resource/William_B._Conway
1,dmrose15,Person,http://dbpedia.org/resource/Albert_Bustamante
2,_richard_ong_,Person,http://dbpedia.org/resource/Baxter_Edwards_Perry
3,_richard_ong_,Person,http://dbpedia.org/resource/William_Millar
4,blizthehusky,Person,http://dbpedia.org/resource/Mark_Buesgens


In [ ]:
new_ents.to_csv('new_entities.csv')

In [ ]:
preds

array([2, 2, 0, ..., 2, 2, 2])

In [ ]:
j = 0
for tweet in tweets['TweetText'].values[0:3]:
  tmp = []
  index = yz_last.index(tweet)
  if preds[index] == 2:
    sampled_list = random.sample(repubs_filter, 1)
    print(tweets['UserName'].values[j]+"\t"+tweet+"\tnegative")
    print(tweets['UserName'].values[j]+"\tPerson\t"+sampled_list[0])

  else:
    sampled_list = random.sample(democs_filter, 1)
    print(tweets['UserName'].values[j]+"\t"+tweet+"\tpositive")
    print(tweets['UserName'].values[j]+"\tPerson\t"+sampled_list[0])

  j=j+1


dmrose15	to many are offended if you simply say yes or no . I have been cold because I ' dont sugar - coat anything '. Honor yes or no , if I am . Most time I do not respond until direct question . Good evening 1	positive
dmrose15	Person	http://dbpedia.org/resource/Ken_Rothman
_richard_ong_	Trump . is truly up . How are going to parade around the idea that Alec kill , but not around the police who shot ?? But Kyle is a patriot ?? loser !! 1	negative
_richard_ong_	Person	http://dbpedia.org/resource/C._W._Bishop
blizthehusky	hey why ' t Kyle in jail ? insider . Trump . is selling that say ' don ' t kill people , Alec people '... fired a prop gun on the " Rust " movie set last week , killing the cinematographer .	negative
blizthehusky	Person	http://dbpedia.org/resource/Phelps_Anderson
